In [1]:
from src.PubmedEndpoint import PubmedEndpoint

publication_text = PubmedEndpoint.fetch_full_text_from_pubmed_id('24212882')

In [2]:
from src.CoordinatesExtractionService import CoordinatesExtractionService

coordinates_extraction_service = CoordinatesExtractionService('gpt-4o-mini')
coordinates_list = coordinates_extraction_service.extract_coordinates_from_text(publication_text)
coordinates_list

/home/wojtek/anaconda3/envs/enhancerlit/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(
/home/wojtek/anaconda3/envs/enhancerlit/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(
/home/wojtek/anaconda3/envs/enhancerlit/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_depr

Regex match score: [False, False, False, False, False]


['chr10:23508437A>G',
 'chr10:23508363A>G',
 'chr10:23508305A>G',
 'chr10:23508365A>G/chr10:23508446A>C',
 'chr10:23502416-23510031']

In [3]:
from src.ContextExtractionService import ContextExtractionService

context_extraction_service = ContextExtractionService('gpt-4o-mini')
coordinate = coordinates_list[3]
coordinate_context = context_extraction_service.extract_context_from_coordinate(publication_text, coordinate)
print(coordinate_context)

The following sentences from the publication are relevant to the genomic coordinates chr10:23508365A>G and chr10:23508446A>C:

1. "We also identified a homozygous mutation in one patient with pancreatic agenesis and intrahepatic cholestatic failure."
2. "Of the 10 probands with mutations in this element, 6 had the same chr10:23508437A>G mutation as part of a shared extended haplotype (minimal shared haplotype of 1.2Mb)."
3. "Three of the remaining probands had different base substitution mutations: a homozygous chr10:23508363A>G mutation, a homozygous chr10:23508305A>G mutation and compound heterozygous chr10:23508365A>G/chr10:23508446A>C mutations."
4. "Testing of parents and siblings demonstrated co-segregation of the mutations with diabetes and exocrine insufficiency."

These sentences provide context about the specific mutations at the given coordinates, their occurrence in patients with pancreatic agenesis, and the analysis of genetic inheritance related to these mutations.


In [5]:
import importlib
import src.SequenceOntologyMappingService  # Initial import
importlib.reload(src.SequenceOntologyMappingService)  # Reload the module
from src.SequenceOntologyMappingService import SequenceOntologyMappingService  # Re-import the class

sequence_ontology_mapping_service = SequenceOntologyMappingService('gpt-4o-mini')
sequence_ontology_term = sequence_ontology_mapping_service.map_coordinates_to_sequence_ontology(coordinate, coordinate_context)
print(sequence_ontology_term)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


```json
{"name": "minus_1_frameshift_variant", "definition": "\"A sequence variant which causes a disruption of the translational reading frame, by shifting one base ahead.\" [http://arjournals.annualreviews.org/doi/pdf/10.1146/annurev.ge.08.120174.001535]", "comment": null, "_distance": 156.83042907714844}
```


In [17]:
import src.LinksFromQueryExtractionService
importlib.reload(src.LinksFromQueryExtractionService)
from src.LinksFromQueryExtractionService import LinksFromQueryExtractionService

links_from_query_extraction_service = LinksFromQueryExtractionService('gpt-4o-mini')
user_query_dict = {'type of variant': 'frameshift'}
links = links_from_query_extraction_service.extract_links_from_query(coordinate, coordinate_context, sequence_ontology_term, user_query_dict)

Link between type of variant and frameshift: False. Type of relation is RELATION_TYPE  . Reasoning: The context indicates that the mutations listed (including chr10:23508365A>G and chr10:23508446A>C) do not specifically refer to a frameshift variant. Instead, the provided sequence ontology describes a "minus_1_frameshift_variant," which is a specific type of sequence variant that disrupts the translational reading frame. There is no direct evidence in the context provided that links the user value "frameshift" specifically to the mentioned coordinates as a confirmed type of variant.


In [10]:
coordinate_context

'The following sentences from the publication are relevant to the genomic coordinates chr10:23508365A>G and chr10:23508446A>C:\n\n1. "We also identified a homozygous mutation in one patient with pancreatic agenesis and intrahepatic cholestatic failure."\n2. "Of the 10 probands with mutations in this element, 6 had the same chr10:23508437A>G mutation as part of a shared extended haplotype (minimal shared haplotype of 1.2Mb)."\n3. "Three of the remaining probands had different base substitution mutations: a homozygous chr10:23508363A>G mutation, a homozygous chr10:23508305A>G mutation and compound heterozygous chr10:23508365A>G/chr10:23508446A>C mutations."\n4. "Testing of parents and siblings demonstrated co-segregation of the mutations with diabetes and exocrine insufficiency."\n\nThese sentences provide context about the specific mutations at the given coordinates, their occurrence in patients with pancreatic agenesis, and the analysis of genetic inheritance related to these mutations